이미지에서 개체's 검출하여 바운딩 박스 그리기

In [19]:
from io import BytesIO
import requests
from PIL import ImageFont
import platform
import random
import gradio as gr
import os
from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 
def get_font():
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            # 윈도우: 맑은 고딕
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            # 맥: 애플 고딕
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux 등
            # 기본 폰트 (한글 지원 안 될 수 있음)
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 지정한 폰트 파일이 없을 경우 PIL 기본 폰트 사용
        return ImageFont.load_default()


# 이미지에서, 개체를 검출하여 개체에 대한 좌표 및 크기를 그려줌
def request_image_analysis(image_url, features_checkbox, options) : 
    endpoint = "https://8ai022-vision-foundry929592821636.cognitiveservices.azure.com/computervision/imageanalysis:analyze"

    params = {
        "features" : ",".join(features_checkbox),
        "api-version" : "2024-02-01"
    }

    # 성별 중립, smartCrop 옵션 추가
    params.update(options)

    headers = {
        "Ocp-Apim-Subscription-Key" : os.getenv("AZURE_VISION_KEY")
    }
    body = {
        "url" : image_url
    }

    response = requests.post(endpoint, headers=headers, params=params, json=body)

    if response.status_code != 200 :
        print(f"Http Request Error {response.status_code}")
        return None
    
    return response.json()


# 파이썬 이미지 라이브러리
from PIL import Image, ImageDraw
def draw_image(image_url, data, features_checkbox) : 

    # 외부 이미지 가져오기
    response = requests.get(image_url)

    # 바이너리 형태 이미지를 열기 위해, BytesIO 사용
    image = Image.open(BytesIO(response.content)) # 이미지 PIL 형태로 변환
    draw = ImageDraw.Draw(image)                  # PIL 그릴 수 있는 형태로 변환
    
    if "objects" in features_checkbox : 

        block_list = data.get("objectsResult").get("values")

        # 이미지에, 빨간색으로 영역 그리기
        for block in block_list : 
            color = random_color()
            boundingBox = block.get("boundingBox")                              # 바운딩 박스
            name = block.get("tags")[0]['name']                                 # 개체명
            confidence = block.get("tags")[0]['confidence']                     # 신뢰도

            if confidence < 0.7 :  # 신뢰도가 70% 미만인 경우 무시
                continue

            x, y, w, h = boundingBox['x'], boundingBox['y'], boundingBox['w'], boundingBox['h']

            draw.rectangle(
                [(x, y), (x + w, y + h)],   # 왼쪽 위, 오른쪽 아래 좌표
                outline = color,
                width = 3
            )
            
            # 좌측 하단에 표기
            draw.text(
                (x, y + h),
                f"개체명 : {name}, 신뢰도 : {confidence:.2f}%",
                fill = color,
                font=get_font()
            )
            

    if "caption" in features_checkbox :
        caption_text = data.get("captionResult").get("text")

        draw.text(
                (0, 30),
                f"Caption : {caption_text}",
                fill = random_color(),
                font=get_font()
            )
        
    if "denseCaptions" in features_checkbox :
        block_list = data.get("denseCaptionsResult").get("values")

        # 이미지에, 빨간색으로 영역 그리기
        for block in block_list : 
            color = random_color()
            boundingBox = block.get("boundingBox")                   # 바운딩 박스
            text = block.get("text")                                 # 개체명
            confidence = block.get("confidence")                     # 신뢰도

            if confidence < 0.7 :  # 신뢰도가 70% 미만인 경우 무시
                continue

            x, y, w, h = boundingBox['x'], boundingBox['y'], boundingBox['w'], boundingBox['h']

            draw.rectangle(
                [(x, y), (x + w, y + h)],   # 왼쪽 위, 오른쪽 아래 좌표
                outline = color,
                width = 3
            )
            
            draw.text(
                (x, y + h + 20),
                f"denseCaption : {text}, 신뢰도 : {confidence:.2f}%",
                fill = color,
                font=get_font()
            )
    
    if "smartCrops" in features_checkbox :
        block_list = data.get("smartCropsResult").get("values")

        for block in block_list : 
            boundingBox = block.get("boundingBox")
            aspectRatio = block.get("aspectRatio")
            color = random_color()
            
            x, y, w, h = boundingBox['x'], boundingBox['y'], boundingBox['w'], boundingBox['h']
            
            draw.rectangle(
                [(x, y), (x + w, y + h)],   # 왼쪽 위, 오른쪽 아래 좌표
                outline = color,
                width = 3
            )
            
            draw.text(
                (x, y + h + 20),
                f"smartCrops : {aspectRatio}",
                fill = color,
                font=get_font()
            )


    # 이미지 저장
    import datetime
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    save_path = f'files/detected_box_{now}.png'
    image.save(save_path)

    # 파일 경로 반환
    return save_path

    



with gr.Blocks() as demo : 
    def click_send(image_url, features_checkbox, is_neutral, ratio_text) : 

        if len(features_checkbox) == 0 :
            return None, []
        

        # request param에 추가할 옵션 정보 설정
        options = dict()
        if "caption" in features_checkbox or "denseCaptions" in features_checkbox :
            options.update({
                "gender-neutral-caption" : is_neutral    # 성별 중립 여부
            })
        if "smartCrops" in features_checkbox :
            options.update({
                "smartcrops-aspect-ratios" : ratio_text  # smartCrop 비율
            })

        # 이미지 개체 검출 요청 
        response = request_image_analysis(image_url, features_checkbox, options)
        
        # 이미지에 개체 바운딩 박스 그리기
        save_path = draw_image(image_url, response, features_checkbox)

        # 파일 경로, 결과 JSON 반환
        return save_path, response

    def change_features(features_checkbox, is_neutral, ratio_text) :
        # features_checkbox에 "caption" 또는 "denseCaptions"가 포함되어 있으면, 성별 중립 옵션을 보이도록 설정
        return_val1 = False
        # features_checkbox에, "smartCrops"가 포함되어 있으면, ratio_text를 보이도록 설정
        return_val2 = False

        if "caption" in features_checkbox or "denseCaptions" in features_checkbox :
            return_val1 = True
        if "smartCrops" in features_checkbox :
            return_val2 = True

        # 컴포넌트 업데이트 처리. (outputs에 정의된 순서대로 반환)        
        return gr.update(visible=return_val1), gr.update(visible=return_val2)
            
        
    gr.Markdown("## Image Analysus")

    """
    obejcts : 이미지 내 객체 검출
    caption : 이미지 캡션 생성
    denseCaptions : 이미지 내 영역별 캡션
    read : 이미지 내 텍스트 인식
    tags : 이미지 태그 생성
    smartCrops : 이미지 내 주요 영역 크롭
    """
    FEATURES = ["objects", "caption", "denseCaptions", "read", "tags", "smartCrops"]
    features_checkbox = gr.CheckboxGroup(label="기능 선택", choices=FEATURES)


    # 선택 FEATURES에 따라, 화면에 보여지거나 숨기기
    is_neutral = gr.Radio(label="성별 중립", choices=[("중립", True), ("구분", False)], value=False, visible=False)
    ratio_text = gr.Textbox(label="smartCrops 크기", placeholder="EX) 0.75, 1.2, 1.8", visible=False)

    image_url_text = gr.Textbox(label="Image URL")      # 이미지 URL 입력 텍스트 박스
    send_button = gr.Button("전송")

    with gr.Row() as r: 
        output_image = gr.Image(label="결과 이미지", type="filepath") # 만약 click_send의 반환값이 파일이면, type="pil"로 지정해야 함.
        result_json = gr.JSON(label="결과 데이터")                    # 결과 JSON 출력

    send_button.click(fn=click_send, inputs=[image_url_text, features_checkbox, is_neutral, ratio_text], outputs=[output_image, result_json])
    features_checkbox.change(fn=change_features, inputs=[features_checkbox, is_neutral, ratio_text], outputs=[is_neutral, ratio_text])

demo.launch()


#send("https://cdn.pixabay.com/photo/2023/01/09/17/49/people-7707981_1280.jpg")

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.
